In [2]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

In [3]:
modelo_olimpiada = BayesianModel([('Genetica','TesteOlimpico'),
                                 ('Pratica','TesteOlimpico'),
                                 ('TesteOlimpico','Oferta')])

# Inicializar relações

In [4]:
genetics_cpd = TabularCPD(
                         variable='Genetica',
                         variable_card=2,
                         values = [[.2,.8]])

In [5]:
practice_cpd = TabularCPD(
                         variable = 'Pratica',
                         variable_card=2,
                         values=[[.7,.3]])

In [6]:
offer_cpd = TabularCPD(
                      variable='Oferta',
                      variable_card=2,
                      values=[[.95, .8, .5],
                             [.05,.2,.5]],
                      evidence=['TesteOlimpico'],
                      evidence_card=[3])

In [7]:
olympics_trials_cpd = TabularCPD(
                                 variable='TesteOlimpico',
                                 variable_card=3,
                                 values = [[.5,.8,.8,.9],
                                          [.3,.15,.1,.08],
                                          [.2,.05,.1,.02]],
                                 evidence=['Genetica', 'Pratica'],
                                 evidence_card = [2,2])

# Adicinando as relações para o modelo

In [8]:
modelo_olimpiada.add_cpds(genetics_cpd, practice_cpd, offer_cpd, olympics_trials_cpd)

# Examina a estrutura do grafo

In [9]:
modelo_olimpiada.get_cpds()

[<TabularCPD representing P(Genetica:2) at 0x7ff60dff25f8>,
 <TabularCPD representing P(Pratica:2) at 0x7ff60dff26a0>,
 <TabularCPD representing P(Oferta:2 | TesteOlimpico:3) at 0x7ff60dff29b0>,
 <TabularCPD representing P(TesteOlimpico:3 | Genetica:2, Pratica:2) at 0x7ff60dff2ba8>]

# Passo de influência

In [10]:
modelo_olimpiada.active_trail_nodes('Pratica')

{'Pratica': {'Oferta', 'Pratica', 'TesteOlimpico'}}

In [11]:
modelo_olimpiada.active_trail_nodes('TesteOlimpico')

{'TesteOlimpico': {'Genetica', 'Oferta', 'Pratica', 'TesteOlimpico'}}

# Independências

In [12]:
modelo_olimpiada.local_independencies('Genetica')

(Genetica _|_ Pratica, TesteOlimpico, Oferta)

In [13]:
modelo_olimpiada.get_independencies()

(Genetica _|_ Pratica)
(Genetica _|_ Oferta | TesteOlimpico)
(Genetica _|_ Oferta | Pratica, TesteOlimpico)
(Pratica _|_ Genetica)
(Pratica _|_ Oferta | TesteOlimpico)
(Pratica _|_ Oferta | TesteOlimpico, Genetica)
(Oferta _|_ Pratica, Genetica | TesteOlimpico)
(Oferta _|_ Pratica | TesteOlimpico, Genetica)
(Oferta _|_ Genetica | Pratica, TesteOlimpico)

# Fazer inferência

Fazer inferencia não explícita

In [14]:
from pgmpy.inference import VariableElimination

In [15]:
inferencia_olimpiada = VariableElimination(modelo_olimpiada)

In [16]:
prob_offer = inferencia_olimpiada.query(variables=['Oferta'])
print(prob_offer['Oferta'])

╒══════════╤═══════════════╕
│ Oferta   │   phi(Oferta) │
╞══════════╪═══════════════╡
│ Oferta_0 │        0.8898 │
├──────────┼───────────────┤
│ Oferta_1 │        0.1102 │
╘══════════╧═══════════════╛


# Conseguir oferta sabendo que a pessoa tem genes ruim

In [17]:
prob_offer_bad_genes = inferencia_olimpiada.query(
                                                 variables=['Oferta'],
                                                 evidence={'Genetica':1})
print(prob_offer_bad_genes['Oferta'])

╒══════════╤═══════════════╕
│ Oferta   │   phi(Oferta) │
╞══════════╪═══════════════╡
│ Oferta_0 │        0.9017 │
├──────────┼───────────────┤
│ Oferta_1 │        0.0983 │
╘══════════╧═══════════════╛


# Conseguir oferta sabendo que a pessoa tem genes bom

In [18]:
prob_offer_good_genes = inferencia_olimpiada.query(
                                                 variables=['Oferta'],
                                                 evidence={'Genetica':0})
print(prob_offer_good_genes['Oferta'])

╒══════════╤═══════════════╕
│ Oferta   │   phi(Oferta) │
╞══════════╪═══════════════╡
│ Oferta_0 │        0.8420 │
├──────────┼───────────────┤
│ Oferta_1 │        0.1580 │
╘══════════╧═══════════════╛


# Conseguir oferta sabendo que a pessoa tem genes ruim e praticou

In [19]:
prob_offer_good_genes_praticou = inferencia_olimpiada.query(
                                                     variables=['Oferta'],
                                                     evidence={'Genetica':0,
                                                               'Pratica':0})
print(prob_offer_good_genes_praticou['Oferta'])

╒══════════╤═══════════════╕
│ Oferta   │   phi(Oferta) │
╞══════════╪═══════════════╡
│ Oferta_0 │        0.8150 │
├──────────┼───────────────┤
│ Oferta_1 │        0.1850 │
╘══════════╧═══════════════╛


# Probabilidade dos genes da pessoa sabendo que ela foi ótima no teste

In [20]:
prob_good_genes_se_otimo_olympic_trials = inferencia_olimpiada.query(
                                                                    variables=['Genetica'],
                                                                    evidence = {'TesteOlimpico':2})
print(prob_good_genes_se_otimo_olympic_trials['Genetica'])

╒════════════╤═════════════════╕
│ Genetica   │   phi(Genetica) │
╞════════════╪═════════════════╡
│ Genetica_0 │          0.3377 │
├────────────┼─────────────────┤
│ Genetica_1 │          0.6623 │
╘════════════╧═════════════════╛


# Probabilidade dos genes da pessoa sabendo que ela não praticou

In [21]:
prob_good_genes_naopraticou = inferencia_olimpiada.query(
                                                        variables=['Genetica'],
                                                        evidence={'Pratica':1})
print(prob_good_genes_naopraticou['Genetica'])

╒════════════╤═════════════════╕
│ Genetica   │   phi(Genetica) │
╞════════════╪═════════════════╡
│ Genetica_0 │          0.2000 │
├────────────┼─────────────────┤
│ Genetica_1 │          0.8000 │
╘════════════╧═════════════════╛


# Probabilidade dos genes da pessoa sabendo que ela não praticou e foi ótima no teste

In [22]:
prob_good_genes_naopraticou_se_bom_olympic_trials = inferencia_olimpiada.query(
                                                        variables=['Genetica'],
                                                        evidence={'Pratica':1,
                                                                 'TesteOlimpico':2})
print(prob_good_genes_naopraticou_se_bom_olympic_trials['Genetica'])

╒════════════╤═════════════════╕
│ Genetica   │   phi(Genetica) │
╞════════════╪═════════════════╡
│ Genetica_0 │          0.3846 │
├────────────┼─────────────────┤
│ Genetica_1 │          0.6154 │
╘════════════╧═════════════════╛


## Estado mais provável

In [23]:
inferencia_olimpiada.map_query(variables = ['Pratica'])

{'Pratica': 0}

In [24]:
inferencia_olimpiada.map_query(variables = ['Oferta'])

{'Oferta': 0}